In [1]:
import csv
from itertools import repeat
import os
import pandas as pd
from pandas import read_csv, DataFrame, Index, MultiIndex
import copy
from IPython.display import display
import numpy as np
import collections
from itertools import dropwhile
import os
from subprocess import call

## - Set up Environment

In [2]:
#----------------------------#
# Configure Environment
#----------------------------#
path = '/autofs/space/lilli_001/users'
NARC = os.path.join('%s/DARPA-ARC_RER' % path)
NARC_analysis = os.path.join('%s/DARPA-Scripts/tutorials/darpa_pipelines_EH/darpa_newARC_scripts/params' % path)

transfer_scanlist=[]
subject_list=[]

unpack_log='unpack.log'
raw_cfg= 'cfg_raw.txt'
new_cfg='cfg.txt'

#----------------------------#
# Generate Step 1,2 C shells
#----------------------------#
# os.system('cd %s; touch subjlist.txt scan.txt unpacksdcmdir_step1.csh unpacksdcmdir_step2.csh' % NARC_analysis)

## - Generate Subject and Scan lists

In [3]:
#----------------------------#
# Import *ALL* scans from BOURGET
#----------------------------#
# Automatically generate FINDSESS.txt, subjlist.txt, scan.txt

#os.system('findsession -p bfr > %s/FINDSESS.txt' % NARC_analysis)

# def get_scans_subjects(path):
#     with open('%s/FINDSESS.txt' % path ,'r') as in_txt, \
#         open('%s/subjlist.txt' % path ,'w') as sublist, \
#         open('%s/scan.txt' % path ,'w') as scanlist:
#         for line in in_txt:
#                 if 'SUBJECT' in line:
#                     sub=line.strip('SUBJECT:  ')
#                     sublist.write('%s \n' % sub.strip())
#                     subject_list.append(sub)
#                 elif 'PATH' in line:
#                     scan=line.strip('PATH   :')
#                     scanlist.write('%s \n' % scan.strip())
#                     transfer_scanlist.append(sub)
#     return
#
# get_scans_subjects(NARC)

#----------------------------#
# Import *SPECIFIC* scans from BOURGET
#----------------------------#
# Manually modify subjlist.txt, scan.txt

def get_scans_subjects(analysis_path,scan_list,subjs_list):
    with open('%s/subjlist.txt' % analysis_path ,'r') as sublist, \
    open('%s/scan.txt' % analysis_path ,'r') as scanlist:
        for subject,scan in zip(sublist,scanlist):
            subjs_list.append(subject.strip())
            scan_list.append(scan.strip())
    return

get_scans_subjects(NARC_analysis,transfer_scanlist,subject_list)

## - unpackdcmsdir - 1

In [ ]:
def unpacksdcmdir_step1(analysis_path,scan_list,subjs_list,raw_data_path):
    with open('%s/unpacksdcmdir_step1.csh' % analysis_path ,'w') as unpacksdcmdir_1:
        for scan,subject in zip(scan_list,subjs_list):
            unpacksdcmdir_1.write('unpacksdcmdir -src %s -targ %s/%s -scanonly %s/%s/%s;\n' \
            % (scan, raw_data_path, subject, raw_data_path, subject, new_cfg))
        print "Next: manually source unpacksdcmdir_step1.csh"
    return

unpacksdcmdir_step1(NARC_analysis,transfer_scanlist,subject_list,NARC)

In [ ]:
#----------------------------#
# Fix this cell to correctly 
# source FreeSurfer6
#----------------------------#

# prefix w/ ! OR %% bash

# cd $lilli_001/DARPA-Scripts/tutorials/darpa_pipelines_EH/darpa_newARC_scripts/params; \
# step_1_path=`pwd`; \
# echo 'Changed directory to... ' `pwd`; \
# echo 'Sourcing FreeSurfer'; \
# set FREESURFER_HOME=/usr/local/freesurfer; \
# export SUBJECTS_DIR=$lilli_001/DARPA-Scripts/tutorials/darpa_pipelines_EH/darpa_newARC_scripts/; \
# source $FREESURFER_HOME/nmr-stable60-env
# echo 'Sourcing unpacksdcmdir_step1.csh'; \
# echo ${step_1_path}/unpacksdcmdir_step1.csh; \
# echo 'Done: unpacksdcmdir_step1.csh';

## - Create Configuration File

In [8]:
#----------------------------#
# START HERE - DEBUG
#----------------------------#

x2={'a':[],'b':[],'c':[],'d':[]}
def read_write_cfg(raw_data_path,subjs_list,unpack_log,raw_cfg): 
    for subject in subjs_list:
        infile=os.path.join(raw_data_path,subject,unpack_log)
        outfile=os.path.join(raw_data_path,subject,raw_cfg)
        with open(infile,'r') as in_txt, open(outfile,'w') as out_txt:
            for line in in_txt:
                if 'ok' in line:
                    c=line.strip().split()
                    n=str(c[0])
                    scan=str(c[1])
                    num=str(c[7])
                    x2['a'].append(n)
                    nifti='NIFTI'
                    x2['b'].append(scan)
                    x2['c'].append(nifti)
                    x2['d'].append(num)
    return

read_write_cfg(NARC,subject_list,unpack_log,raw_cfg)

In [ ]:
#----------------------------#
# START HERE - DEBUG
#----------------------------#

def format_cfg(raw_data_path,subjs_list,raw_cfg,new_cfg):
    for subject in subjs_list:
        subdir = os.path.join(raw_data_path,"%s/cfg.txt" % subject.rstrip(' \n'))
        with open(subdir, 'w') as form_cfg:
            for a,b,c,d in zip(x2['a'],x2['b'],x2['c'],x2['d']):
                form_cfg.write('\t'.join([a,b,c,d]) + '\n')
            print 'cfg.txt formatted in subject %s directory' % subject
    return

format_cfg(NARC,subject_list,raw_cfg,new_cfg)

## - unpackdcmsdir - 2

In [ ]:
with open('%s/unpacksdcmdir_2.csh' % NARC ,'w') as unpacksdcmdir_2, \
open('%s/subjlist.txt' % NARC ,'r') as sublist:
    subdir = os.path.join(NARC,"%s/cfg.txt" % subject.rstrip(' \n'))
    for scan,subject in zip(transfer_scanlist,sublist):        
        unpacksdcmdir_2.write('unpacksdcmdir -src %s -targ %s/%s -fsfast -cfg %s \n'\
        % (scan.rstrip(' \n'), NARC.rstrip(' \n'), subject.rstrip(' \n'), subdir))
    print "Next: source unpacksdcmdir_2.csh"

## APPENDIX / MISC

### DICOM to NIFTI

In [ ]:
scan_list=['fieldmap_BOLD','AAScout', 'DIFFUSION_HighRes_REVISED', 'fieldmap_BOLD', \
           'fieldmap_generic_DTI', 'fMRI_ASSOCIATIVE_BOLD', 'fMRI_resting_state_1', \
           'fMRI_resting_state_2', 'fMRI_REWARD_BOLD', 'localizer', 'mprage_1mm_iso']

output=[]

def dicom2nii_get_path(path):
    for subject in subject_list:
        if subject in os.listdir(path):
            for scan in scan_list:
                if scan in os.listdir(os.path.join(path,subject)):
                    insert=os.path.join(path, subject, scan)
                    subdir=os.listdir(insert)
                    for item in subdir:
                        if item.startswith('0'):
                            insert2=os.path.join(path, subject, scan, item)
                            dicom1=os.listdir(insert2)[0]
                            insert3=os.path.join(path, subject, scan, item, dicom1)
                            save_me=output.append(insert3)  
    return

dicom2nii_get_path('/autofs/space/will_001/users/TLC/pilot_1st20subj/MGH/')

### Read Configuration File

In [ ]:
def read_cfg(path,subjid,doc): 
    insert=os.path.join(path, subjid, doc)
    doc=open(insert,'r' )
    print doc.read()
    
read_cfg('/autofs/space/will_001/users/TLC/pilot_1st20subj/MGH/','020043MR01','unpack.log')

### Read unpack.log and write cfg.txt file 
##### Note: Iterates through subjects txt file in lieu of array

In [ ]:
def iter_subjlist(path,subject_list,unpack_log,raw_cfg,new_cfg):
    subjects=os.path.join(path,subject_list)
    with open(subjects,'U') as slist:
        for subject in slist: 
            subject=subject.strip().split()
            print(subject)
            read_write_cfg(path,subject,unpack_log,raw_cfg)
            format_cfg(path,subject,raw_cfg,new_cfg)
    return